In [1]:
%cd ..
%pip install -e .

/home/sandor/dtu/2021-22-spring/advanced_machine_learning/p-reg
Obtaining file:///home/sandor/dtu/2021-22-spring/advanced_machine_learning/p-reg
  Preparing metadata (setup.py) ... done
  Attempting uninstall: src
    Found existing installation: src 0.1.0
    Uninstalling src-0.1.0:
      Successfully uninstalled src-0.1.0
  Running setup.py develop for src
Note: you may need to restart the kernel to use updated packages.


In [2]:
from src.models.dense import NN0
from src.models.dense import NN1
from src.models.gcn import GCN0

from src.models.train_model import train0
from src.models.train_model import set_masks
from src.models.evaluate_model import evaluate0

from src.models.utils import random_splits
from src.models.utils import propagate

import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt

from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
from torch_geometric.datasets import Planetoid

from sklearn.manifold import TSNE

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

dataset = Planetoid(root='data/Planetoid', name='Cora')
data = dataset[0].to(device)

In [4]:
# triaing the model
# for A, B in [[5,2], [10,5], [20, 5], [40, 10], [100, 20]]:
for A, B in [[100, 20]]:
    data = random_splits(data, A, B)

    gcn_model = GCN0(num_node_features=dataset.num_node_features,
                    num_classes=dataset.num_classes) \
                    .to(device)

    gcn_model = train0(gcn_model, data)

    acc = evaluate0(gcn_model, data)
    print(f'Accuracy: {acc:.4f}')


Accuracy: 0.8517


In [5]:
data.y[data.train_mask].numpy().tolist().count(1)

100

In [6]:
data.edge_index.T[0]


tensor([  0, 633])

In [7]:
len(data.x[0])

1433

In [8]:
from tqdm import tqdm

embeddings = gcn_model.conv1(data.x, data.edge_index)
propagated = propagate(data, embeddings)

  4%|▎         | 98/2708 [00:07<03:29, 12.46it/s]


KeyboardInterrupt: 

In [ ]:
print(embeddings[1])

tensor([-0.3566, -0.4466,  1.3126, -0.2468, -0.3643,  1.7459,  1.6251, -0.6035,
        -0.5813,  1.7080, -0.3912,  2.0018, -0.5811, -0.8225,  3.1170,  2.1119],
       grad_fn=<SelectBackward0>)


In [ ]:
print(propagated[1])

tensor([-0.2726, -0.4046,  0.9721,  0.1953, -0.1561,  1.0718,  1.0694, -0.1053,
        -0.2970,  1.3685, -0.4097,  1.6113, -0.2383, -0.2017,  2.2109,  1.4837],
       grad_fn=<SelectBackward0>)
